In [1]:
import torch
import numpy as np
import torchvision
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn.functional as F

In [2]:
class Network(nn.Module):
    
    def __init__(self):

        super().__init__()

        # sees 32, 32, 3
        self.conv1 = nn.Conv2d(3, 64, 11, padding=5)
        
        # sees 16, 16, 64
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        
        # sees 16, 16, 128
        self.conv3 = nn.Conv2d(128, 128, 3, padding=1)
        
        # will get flattened to 128
        self.fc1 = nn.Linear(128, 10)
    
        # sees 16, 16, 128 -> will flatten to 128
        self.global_max_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.max_pool = nn.MaxPool2d(2, 2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        x = self.max_pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        # global max pool
        x = self.global_max_pool(x)
        
        # flatten (remove the 1 dimensions)
        x = torch.squeeze(x)
        
        x = F.relu(self.fc1(x))
        x = self.softmax(x)

        return x

In [3]:
# load CIFAR image data

In [4]:
BATCH_SIZE = 50

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# create network object
model = Network()

In [8]:
# define the optimizer and loss function

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Device configuration
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = 'cpu'

In [ ]:
# train the model
epochs = 50
total_steps = len(trainloader)
training_losses = []
test_losses = []
train_acc = []
test_acc = []

# loop through epochs
for epoch in range(epochs):
    
    train_running_loss = 0  # track train running loss
    correct = 0
    total = 0
    
    # load batch images/labels
    for step, (images, labels) in enumerate(trainloader):
        
        # put data onto available device
        images = images.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
    
        outputs = model(images)  # forward pass
        _, predicted = torch.max(outputs.data, 1)  # retrieve top preds
        
        total += labels.size(0)  # add batch size
        correct += (predicted == labels).sum().item()  # calc num correct
        
        loss = criterion(outputs, labels)  # calc loss
        train_running_loss += loss.item()  # acc running loss
        
        loss.backward()   # backprop
        optimizer.step()  # forward

        if step % 100 == 0:  # print progress by iteration
        
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
            .format(epoch+1, epochs, step+1, total_steps, loss.item()))
    
    # div by num batches to get average
    epoch_train_loss = train_running_loss / len(trainloader)
                   
    print('Epoch [{}/{}], Train Loss: {:.4f}'.format(epoch+1, epochs, epoch_train_loss))
        
    # append the loss/acc after all the steps 
    training_losses.append(epoch_train_loss)
    train_acc.append(correct / total)
        
    
    # ------------------------------ #
    

    # evaluate on test data
    model.eval()
    with torch.no_grad():
        test_running_loss = 0  # track test running loss
        correct = 0
        total = 0

        for images, labels in testloader:
                   
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
                   
            loss = criterion(outputs, labels)
            test_running_loss += loss.item()
                   
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
                 
    # div by num batches
    epoch_test_loss = test_running_loss / len(testloader)
                   
    print('Epoch [{}/{}], Test Loss: {:.4f}'.format(epoch+1, epochs, epoch_test_loss))
        
    # append the loss & acc after all the steps 
    test_losses.append(epoch_test_loss)
    test_acc.append(correct / total)
            
    print('Test Accuracy: {} %'.format(100 * correct / total))

Epoch [1/50], Step [1/1000], Loss: 2.3024
Epoch [1/50], Step [101/1000], Loss: 2.3019
Epoch [1/50], Step [201/1000], Loss: 2.2955
Epoch [1/50], Step [301/1000], Loss: 2.2862
Epoch [1/50], Step [401/1000], Loss: 2.2547
Epoch [1/50], Step [501/1000], Loss: 2.2487
Epoch [1/50], Step [601/1000], Loss: 2.2227
Epoch [1/50], Step [701/1000], Loss: 2.2662
Epoch [1/50], Step [801/1000], Loss: 2.2410
Epoch [1/50], Step [901/1000], Loss: 2.2384
Epoch [1/50], Train Loss: 2.2574
Epoch [1/50], Test Loss: 2.2135
Test Accuracy: 22.69 %
Epoch [2/50], Step [1/1000], Loss: 2.2540
Epoch [2/50], Step [101/1000], Loss: 2.2070
Epoch [2/50], Step [201/1000], Loss: 2.1889
Epoch [2/50], Step [301/1000], Loss: 2.2622
Epoch [2/50], Step [401/1000], Loss: 2.2540
Epoch [2/50], Step [501/1000], Loss: 2.2084
Epoch [2/50], Step [601/1000], Loss: 2.1223
Epoch [2/50], Step [701/1000], Loss: 2.1311
Epoch [2/50], Step [801/1000], Loss: 2.1540
Epoch [2/50], Step [901/1000], Loss: 2.1194
Epoch [2/50], Train Loss: 2.1824
Epo

Epoch [16/50], Step [501/1000], Loss: 2.0039
Epoch [16/50], Step [601/1000], Loss: 2.1101
Epoch [16/50], Step [701/1000], Loss: 1.9825
Epoch [16/50], Step [801/1000], Loss: 1.9100
Epoch [16/50], Step [901/1000], Loss: 2.0027
Epoch [16/50], Train Loss: 1.9811
Epoch [16/50], Test Loss: 1.9804
Test Accuracy: 47.84 %
Epoch [17/50], Step [1/1000], Loss: 1.9438
Epoch [17/50], Step [101/1000], Loss: 1.9675
Epoch [17/50], Step [201/1000], Loss: 1.9535
Epoch [17/50], Step [301/1000], Loss: 1.9960
Epoch [17/50], Step [401/1000], Loss: 2.0680
Epoch [17/50], Step [501/1000], Loss: 1.8842
Epoch [17/50], Step [601/1000], Loss: 2.0697
Epoch [17/50], Step [701/1000], Loss: 1.8581
Epoch [17/50], Step [801/1000], Loss: 2.0632
Epoch [17/50], Step [901/1000], Loss: 2.0320
Epoch [17/50], Train Loss: 1.9768
Epoch [17/50], Test Loss: 1.9816
Test Accuracy: 47.8 %
Epoch [18/50], Step [1/1000], Loss: 1.8546


In [ ]:
def plot_graph(train_losses, test_losses, train_acc, test_acc):
    # plot graph
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train loss")
    plt.plot(test_losses, label="Test loss")
    plt.legend(loc='best')
    plt.title("Loss vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss (Cross entropy)")

    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label="Train Accuracy")
    plt.plot(test_acc, label="Test Accuracy")
    plt.legend(loc='best')
    plt.title("Accuracy vs Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()

In [ ]:
plot_graph(training_losses, test_losses, train_acc, test_acc)

In [ ]:
model

In [ ]:
model.features